In [1]:
import itertools
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
def inverted_bases(cadena):
    complementos = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    complemento_inverso = ''.join(complementos[base] for base in reversed(cadena))
    return complemento_inverso

In [3]:
def overlap(a, b, min_length):
	start = 0
	while True:
		start = a.find(b[:min_length], start)
		if start == -1:
			return 0

		if b.startswith(a[start:]):
			return len(a)-start
		start += 1

In [4]:
def pick_maximal_overlap(reads, k):
	reada, readb = None, None
	best_olen = 0
	for a, b in itertools.permutations(reads, 2):
		olen = overlap(a, b, min_length=k)
		if olen > best_olen:
			reada, readb = a, b
			best_olen = olen
	#print(reada, readb, best_olen)
	return reada, readb, best_olen

def greedy_scs(reads, k):
	read_a, read_b, olen = pick_maximal_overlap(reads, k)
	while olen > 0:
		reads.remove(read_a)
		reads.remove(read_b)
		reads.append(read_a + read_b[-(len(read_b) - olen):])
		read_a, read_b, olen = pick_maximal_overlap(reads, k)
	return ''.join(reads)

In [5]:
def create_graph(reads, k):
	G = nx.DiGraph()
	#for r in reads:
	#	G.add_node(r)
	for a, b in itertools.permutations(reads, 2):
		olen = overlap(a, b, min_length=k)
		#print(a, b, olen)
		if olen >= k:
			G.add_edge(a, b, weight=olen)
	return G

In [6]:
def hamiltonian_path(G):
	path = []
	for node in G.nodes():
		visited = set()
		if hamiltonian_path_recursive(G, node, visited, path):
			return path
	return None

def hamiltonian_path_recursive(G, current_node, visited, path):
	visited.add(current_node)
	path.append(current_node)
	if len(visited) == len(G.nodes()):
		return True
	for neighbor in G.neighbors(current_node):
		if neighbor not in visited:
			if hamiltonian_path_recursive(G, neighbor, visited, path):
				return True
	path.pop()
	visited.remove(current_node)
	return False

def find_superstring_from_path(G, path):
	superstring = path[0]
	for i in range(1, len(path)):
		current_node = path[i]
		previous_node = path[i - 1]
		edge_weight = G[previous_node][current_node]['weight']
		superstring += current_node[edge_weight:]
	return superstring

In [37]:
#reads = ['GTG', 'TGG', 'ATG', 'GGC', 'GCG', 'CGT', 'GCA', 'TGC', 'CAA', 'AAT']
#reads = ['AGTATTGGCAATC', 'AATCGATG', 'ATGCAAACCT', 'CCTTTTGG', 'TTGGCAATCACT']
reads = ['ATCCGTTGAAGCCGCGGGC', 'TTAACTCGAGG', 'TTAAGTACTGCCCG', 'ATCTGTGTCGGG', 'CGACTCCCGACACA', 'CACAGATCCGTTGAAGCCGCGGG', 'CTCGAGTTAAGTA', 'CGCGGGCAGTACTT']
k_graph = 3
k_scs = 3

In [11]:
from itertools import product

In [38]:
min_len = float('inf')
final_reads = []
final_substring = ''

all_combinations = []
for combination in product([False, True], repeat=len(reads)):
    tmp_reads = [read if not reverse else inverted_bases(read) for read, reverse in zip(reads, combination)]
    all_combinations.append(tmp_reads)

for c in all_combinations:
	tmp_reads = c.copy()
	substring = greedy_scs(c, k_scs)
	if len(substring) < min_len:
		final_reads = tmp_reads
		final_substring = substring
		min_len = len(substring)

print ("Final sequences: ", final_reads)
print ("DNA assembled by Greedy SCS:", final_substring, len(final_substring))

Final sequences:  ['ATCCGTTGAAGCCGCGGGC', 'TTAACTCGAGG', 'CGGGCAGTACTTAA', 'CCCGACACAGAT', 'CGACTCCCGACACA', 'CACAGATCCGTTGAAGCCGCGGG', 'TACTTAACTCGAG', 'CGCGGGCAGTACTT']
DNA assembled by Greedy SCS: CGACTCCCGACACAGATCCGTTGAAGCCGCGGGCAGTACTTAACTCGAGG 50


In [32]:
G = create_graph(final_reads, k_graph)
#print("Nodes:", G.nodes)
#print("Edges:", G.edges(data=True))

pos = nx.circular_layout(G)
nx.draw(G, pos, with_labels=True, node_size=1000, node_color="skyblue", font_size=6, font_weight="bold")
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.savefig('graph.png')
plt.close()

In [31]:
ham_path = hamiltonian_path(G)
print("Hamiltonian Path:", ham_path)

superstring = find_superstring_from_path(G, ham_path)
print("DNA assembled by Hamiltonian Path:", superstring, len(superstring))

Hamiltonian Path: ['CGACTCCCGACACA', 'CCCGACACAGAT', 'CACAGATCCGTTGAAGCCGCGGG', 'ATCCGTTGAAGCCGCGGGC', 'CGCGGGCAGTACTT', 'CGGGCAGTACTTAA', 'TACTTAACTCGAG', 'TTAACTCGAGG']
DNA assembled by Hamiltonian Path: CGACTCCCGACACAGATCCGTTGAAGCCGCGGGCAGTACTTAACTCGAGG 50
